# Answer technical questions with Frontier and Open Source models

This tool that takes a technical question, and responds with an explanation. This demo shows how to invoke both OpenAI's Frontier models and Meta's Open Source models, deployed locally with Ollama.

In [2]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good!")
else:
    print("There might be a problem with your API key?")

API key looks good!


In [4]:
# constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [5]:
# set up environment
openai = OpenAI(api_key=api_key)
# first run "ollama run llama3.2" to start the local server
ollama = OpenAI(base_url="http://localhost:11434/v1", api_key="")

In [6]:
sys_prompt_default = "You are a helpful programming assistant."
user_prompt_default = """Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

sys_prompt = input("Enter system prompt: ") or sys_prompt_default
user_prompt = input("Enter user prompt: ") or user_prompt_default

In [7]:
def display_response(model_name, response, stream):
    """Display the response from the model."""
    if stream:
        try:
            res_stream = ""
            display_id = display(Markdown(f"*{model_name} is typing...*"), display_id=True).display_id

            for chunk in response:
                delta = chunk.choices[0].delta.content or ""
                res_stream += delta
                update_display(Markdown(f"*{model_name} is typing...*\n\n{res_stream}"), display_id=display_id)

            update_display(Markdown(f"*{model_name}'s answer:*\n\n{res_stream}"), display_id=display_id)
        except Exception as e:
            update_display(Markdown(f"*Error during streaming:*\n{e}"), display_id=display_id)
    else:
        try:
            display(Markdown(f"*{model_name}'s answer:*\n\n{response.choices[0].message.content}"))
        except Exception as e:
            print(f"Error when trying to get a response from the model:")
            print(f"model name: {model_name}\n{e}")


def ask_model_and_display(client, model_name, sys_prompt, user_prompt, stream=False):
    """Ask the model and display the response, either streamed or not."""
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": sys_prompt},
                {"role": "user", "content": user_prompt}
            ],
            stream=stream
        )
    except Exception as e:
        print(f"Error when trying to get a response from the model:")
        print(f"model name: {model_name}\n{e}")
    else:
        display_response(model_name, response, stream)

In [8]:
# Get gpt-4o-mini to answer, with streaming
ask_model_and_display(openai, MODEL_GPT, sys_prompt, user_prompt, stream=True)

*gpt-4o-mini's answer:*

The provided code is a Python expression that uses a generator with the `yield from` statement to yield values from a set comprehension. Let's break it down step by step:

1. **Set Comprehension**: 
   ```python
   {book.get("author") for book in books if book.get("author")}
   ```
   - This creates a set containing all unique authors from a collection of `books`.
   - It iterates over a list of `books` (which is expected to be an iterable), and for each `book` in this list, it calls `book.get("author")` to get the author's name.
   - The `if book.get("author")` condition ensures that only books with a non-null or non-empty author are included in the set. This means if a book does not have an author listed (`None` or an empty string), it is skipped.

2. **Generator with `yield from`**:
   ```python
   yield from ...
   ```
   - The `yield from` statement is used to yield all values from an iterable. In this case, it is used to yield each unique author obtained from the set comprehension.

### Summary:
Overall, this code snippet is a concise way to gather all unique authors from a list of books and yield them one by one. It's useful for scenarios where you want to process or access each author individually in a lazy manner (i.e., one at a time), rather than creating a complete list of authors in memory. 

### Why Use This Approach:
- **Efficiency**: By using a set comprehension, it automatically handles duplicates (only unique authors are stored).
- **Lazy Evaluation**: Using `yield from` allows for on-the-fly generation of authors, which can be more memory-efficient, especially when dealing with large datasets.
- **Clarity**: The use of comprehensions and `yield from` provides a clear and concise way to write the code, making it easier to understand for those familiar with Python.

In [ ]:
# Llama 3.2 with streaming
ask_model_and_display(ollama, MODEL_LLAMA, sys_prompt, user_prompt, stream=True)

In [ ]:
# Llama 3.2 without streaming
ask_model_and_display(ollama, MODEL_LLAMA, sys_prompt, user_prompt, stream=False)